<a href="https://colab.research.google.com/github/claudelepere/ML_GitHub/blob/main/Prediction_EN_791112_50000_51000_all_thr03_SAMPLES_0523.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###### Error: gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.

Ignore this error as gcsfs is not used.

In [1]:
!pip install -q    accelerate
!pip install -q    huggingface_hub
#!pip install -q    scikit-learn
!pip install -q -U transformers     # Hugging Face library
!pip install -q -U datasets         # Hugging Face library
#!pip install -q    wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 48.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 99.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 100.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 141.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import datetime
import json
import matplotlib.pyplot as plt
import numpy             as np
import os
import pandas            as pd
import shutil
import torch

from contextlib                     import suppress
from datasets                       import DatasetDict
from google.colab                   import files, userdata
from huggingface_hub                import create_repo, hf_hub_download, login, whoami
from sklearn.metrics                import classification_report
from transformers.models.longformer import LongformerTokenizerFast, LongformerForSequenceClassification
from torch.utils.data               import DataLoader

##### TO DO BEFORE RUN
- GIVE HF HUB AND W&B ACCESSES
- UPDATE ID OF HF HUB MODEL
- UPDATE NAME OF ipynb FILE

##### Hugging Face Hub authenticate

In [3]:
os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN")    # Store the key in os.environ
hf_token               = os.environ.get('HF_TOKEN')

login(token=hf_token)

# Check
user = whoami(token=hf_token)
assert user['name'] == 'claudelepere', f"{user['name']} is not claudelepere"

print(f"user: {user}")

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


user: {'type': 'user', 'id': '66ec3d5f61228b02f8780beb', 'name': 'claudelepere', 'fullname': 'Claude Lepère', 'isPro': False, 'avatarUrl': 'https://cdn-avatars.huggingface.co/v1/production/uploads/66ec3d5f61228b02f8780beb/gvnf9pvm2KvE90ETMUQo3.jpeg', 'orgs': [], 'auth': {'type': 'access_token', 'accessToken': {'displayName': 'jobs_token', 'role': 'fineGrained', 'createdAt': '2025-01-04T17:44:35.493Z', 'fineGrained': {'canReadGatedRepos': False, 'global': [], 'scoped': [{'entity': {'_id': '66ec3d5f61228b02f8780beb', 'type': 'user', 'name': 'claudelepere'}, 'permissions': ['repo.content.read', 'repo.write']}]}}}}


##### Configuration

###### Define literal data

In [4]:
# HF Hub repo
repo_id              = "claudelepere/jobs_EN_791112_50000_all_0523"

skills               = 791112

all_rows_low_trEvTe  = 0
all_rows_high_trEvTe = 50000

all_rows_low         = 50000
all_rows_high        = 51000
num_datapoints       = all_rows_high - all_rows_low

max_length           = 1024
batch_size           = 4*8

threshold_tuning     = True

min_threshold        = 0.3 # 0.3: OK: 43 KO: 957 0.25: idem 0.35: idem 0.40: idem 0.40: idem  0.40: OK:  42
min_trEvTe_count     =  70 #  70: 0K: 43 KO: 957   70: idem 70:   idem 0.70: idem  200: idem   500: KO: 958

###### Define input paths

In [5]:
# datasetDict zip and dir (= zip filename without extension)
datasetDict_zip = f"dataset_EN_{skills}_{all_rows_low}_{all_rows_high}_all.zip"
datasetDict_dir = os.path.splitext(datasetDict_zip)[0]

print(f"datasetDict_zip: {datasetDict_zip}")
print(f"datasetDict_dir: {datasetDict_dir}")

# Optimized thresholds json file
optimized_thresholds_json = "optimized_thresholds.json"
print(f"optimized_thresholds_json: {optimized_thresholds_json}")

#TrEvTe_0_50000_all_count_json = "TrEvTe_0_50000_all_count.json"
#Pr_50000_51000_all_count_json = "Pr_50000_51000_all_count.json"
TrEvTe_by_count_gt_0_json                   = "TrEvTe_by_count_gt_0.json"
TrEvTe_by_SkillTypeIdComplexityId_gt_0_json = "TrEvTe_by_SkillTypeIdComplexityId_gt_0.json"
#Pr_by_count_gt_0_json                       = "Pr_by_count_gt_0.json"
#Pr_by_SkillTypeIdComplexityId_gt_0_json     = "Pr_by_SkillTypeIdComplexityId_gt_0.json"

datasetDict_zip: dataset_EN_791112_50000_51000_all.zip
datasetDict_dir: dataset_EN_791112_50000_51000_all
optimized_thresholds_json: optimized_thresholds.json


###### Define output paths

In [6]:
# true_labels - preds comparison file
compare_true_labels_preds_file = "compare_true_labels_preds.txt"
output_txt = "output.txt"

##### Get 4 files into the content directory
* Upload 3 local files
  * dataset_EN_791112_50000_51000_all.zip
  * TrEvTe_by_count_gt_0_json
  * TrEvTe_by_SkillTypeIdComplexityId_gt_0_json
* Download 1 HF Hub file
  * optimized_thresholds_json


In [7]:
# dataset cleaning: check whether the zip file and the directory exist in /content and delete them if they do
with suppress(FileNotFoundError):
    if os.path.isfile(datasetDict_zip):
        os.remove(datasetDict_zip)
        print(f"Deleted '{datasetDict_zip}' in /content.")
    if os.path.isdir(datasetDict_dir):
        shutil.rmtree(datasetDict_dir)
        print(f"Deleted '{datasetDict_dir}' in /content.")

# Upload the files to /content, the current working directory
uploaded = files.upload()

for filename in uploaded.keys():
    print(f"Uploaded file: {filename}")

# dataset: unzip datasetDict_zip file to datasetDict_dir in /content
shutil.unpack_archive(datasetDict_zip, "/content")

# TrEvTe_0_50000_all_count_dict
#with open(TrEvTe_0_50000_all_count_json, 'r') as json_file:
#    TrEvTe_0_50000_all_count_dict = json.load(json_file)
#TrEvTe_all_count_by_count_gt_0_dict                   = TrEvTe_0_50000_all_count_dict["by_count_gt_0"]
#TrEvTe_all_count_by_SkillTypeIdComplexityId_gt_0_dict = TrEvTe_0_50000_all_count_dict["by_SkillTypeIdComplexityId_gt_0"]

with open(TrEvTe_by_count_gt_0_json, 'r') as json_file:
    TrEvTe_by_count_gt_0 = json.load(json_file)
print(f"TrEvTe_by_count_gt_0: {type(TrEvTe_by_count_gt_0)} len={len(TrEvTe_by_count_gt_0)}\n{TrEvTe_by_count_gt_0}")

o = TrEvTe_by_count_gt_0["138"]
print(f"o: {type(o)} len={len(o)}\n{o}")
oo = o["count"]
print(f"oo: {type(oo)} {oo}")

with open(TrEvTe_by_SkillTypeIdComplexityId_gt_0_json, 'r') as json_file:
    TrEvTe_by_SkillTypeIdComplexityId_gt_0 = json.load(json_file)

#with open(Pr_by_count_gt_0_json, 'r') as json_file:
#    Pr_by_count_gt_0 = json.load(json_file)
#print(f"Pr_by_count_gt_0: {type(Pr_by_count_gt_0)} len={len(Pr_by_count_gt_0)}\n{Pr_by_count_gt_0}")
#with open(Pr_by_SkillTypeIdComplexityId_gt_0_json, 'r') as json_file:
#    Pr_by_SkillTypeIdComplexityId_gt_0 = json.load(json_file)


# Pr_50000_51000_all_count_dict
#with open(Pr_50000_51000_all_count_json, 'r') as json_file:
#    Pr_50000_51000_all_count_dict = json.load(json_file)
#Pr_all_count_by_count_gt_0_dict                   = Pr_50000_51000_all_count_dict["by_count_gt_0"]
#Pr_all_count_by_SkillTypeIdComplexityId_gt_0_dict = Pr_50000_51000_all_count_dict["by_SkillTypeIdComplexityId_gt_0"]

# optimized_thresholds
optimized_thresholds_path = hf_hub_download(repo_id=repo_id, filename=optimized_thresholds_json)
print(f"optimized_thresholds_path: {type(optimized_thresholds_path)} len={len(optimized_thresholds_path)}\n{optimized_thresholds_path}")
with open(optimized_thresholds_path, 'r') as json_file:
    optimized_thresholds_dict = json.load(json_file)
print(f"optimized_thresholds_dict: {type(optimized_thresholds_dict)} {len(optimized_thresholds_dict)} {optimized_thresholds_dict}")

for label, optimized_threshold in optimized_thresholds_dict.items():
    print(f"{label}: {optimized_threshold:.2f}")


Saving dataset_EN_791112_50000_51000_all.zip to dataset_EN_791112_50000_51000_all.zip
Saving TrEvTe_by_SkillTypeIdComplexityId_gt_0.json to TrEvTe_by_SkillTypeIdComplexityId_gt_0.json
Saving TrEvTe_by_count_gt_0.json to TrEvTe_by_count_gt_0.json
Uploaded file: dataset_EN_791112_50000_51000_all.zip
Uploaded file: TrEvTe_by_SkillTypeIdComplexityId_gt_0.json
Uploaded file: TrEvTe_by_count_gt_0.json
TrEvTe_by_count_gt_0: <class 'dict'> len=147
{'138': {'count': 45021, 'Value': 'EXPERIENCE_MORE_THAN_5_YEARS', 'ShortValue': '5y-10y'}, '139': {'count': 38840, 'Value': 'EXPERIENCE_MORE_THAN_10_YEARS', 'ShortValue': '>10y'}, '137': {'count': 35796, 'Value': 'EXPERIENCE_2_TO_5_YEARS', 'ShortValue': '2y-5y'}, '352': {'count': 17701, 'Value': 'High school', 'ShortValue': None}, '142': {'count': 16712, 'Value': 'Full-stack Developer', 'ShortValue': None}, '136': {'count': 11057, 'Value': 'EXPERIENCE_6_MONTHS_TO_2_YEARS', 'ShortValue': '6m-2y'}, '149': {'count': 10153, 'Value': 'Functional Analyst',

optimized_thresholds.json:   0%|          | 0.00/4.40k [00:00<?, ?B/s]

optimized_thresholds_path: <class 'str'> len=157
/root/.cache/huggingface/hub/models--claudelepere--jobs_EN_791112_50000_all_0523/snapshots/339b2e326cc630c6684006456640bc4e8ad55cb0/optimized_thresholds.json
optimized_thresholds_dict: <class 'dict'> 147 {'142': 0.49999999999999994, '143': 0.44999999999999996, '146': 0.44999999999999996, '147': 0.44999999999999996, '149': 0.44999999999999996, '150': 0.49999999999999994, '152': 0.39999999999999997, '154': 0.49999999999999994, '155': 0.49999999999999994, '156': 0.44999999999999996, '158': 0.44999999999999996, '160': 0.44999999999999996, '162': 0.44999999999999996, '165': 0.49999999999999994, '167': 0.44999999999999996, '169': 0.44999999999999996, '170': 0.44999999999999996, '171': 0.44999999999999996, '173': 0.39999999999999997, '174': 0.49999999999999994, '175': 0.49999999999999994, '176': 0.49999999999999994, '356': 0.49999999999999994, '360': 0.5499999999999999, '361': 0.39999999999999997, '362': 0.39999999999999997, '364': 0.4499999999

##### Checks

In [8]:
!python -V

print(f"currentdir: {os.getcwd()}")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # cuda = A100 GPU or L4 GPU
print(f"device: {device}")

Python 3.11.12
currentdir: /content
device: cuda


##### Out Of Memory (OOM)

###### OOM: check for and kill zombie processes

In [9]:
!ps aux | grep python
!kill -9 <PID>
if torch.cuda.is_available():
    !nvidia-smi
    print(torch.cuda.memory_summary())

root          85  2.0  0.0      0     0 ?        Z    20:24   0:13 [python3] <defunct>
root          86  0.0  0.0  76344 57680 ?        S    20:24   0:00 python3 /usr/local/bin/colab-fileshim.py
root         131  0.6  0.1 884500 132188 ?       Sl   20:25   0:04 /usr/bin/python3 /usr/local/bin/jupyter-notebook --debug --transport="ipc" --ip=172.28.0.12 --NotebookApp.token= --port=9000 --FileContentsManager.root_dir=/ --MappingKernelManager.root_dir=/content
root        2176 10.9  1.5 12417000 1319016 ?    Ssl  20:33   0:17 /usr/bin/python3 -m colab_kernel_launcher -f /root/.local/share/jupyter/runtime/kernel-b01b0b18-b9c6-477d-aa82-ecff448fab74.json
root        2214  0.4  0.0 545908 20952 ?        Sl   20:33   0:00 /usr/bin/python3 /usr/local/lib/python3.11/dist-packages/debugpy/adapter --for-server 47729 --host 127.0.0.1 --port 40009 --server-access-token 0c24f7475908d8fc4e9f4abcf270fa9a5e3f4e7b3b15241b4d928275149896a6
root        2812  0.1  0.0 1243364 19960 ?       Sl   20:34   0:00 

###### OOM: free up GPU memory

In [10]:
torch.cuda.empty_cache()

##### Create datasetDict (HF DatasetDict) = 1 HF Dataset prediction
The sequence of datapoints of dataset=datasetDict['prediction'] is the same as the sequence of SELECT, id, ... WHERE in_LanguageId=1 ORDER BY id DESC in MySQL.

In [11]:
datasetDict = DatasetDict.load_from_disk(datasetDict_dir)
dataset     = datasetDict['prediction']

print(f"dataset: {type(dataset)} shape={dataset.shape}\n{dataset}")     # <class 'datasets.arrow_dataset.Dataset'> shape=(100, 8)
print(f"dataset.features: {type(dataset.features)} shape={dataset.features}\n{dataset.features}")

# Convert the dataset to a pandas dataframe
df_original = pd.DataFrame(dataset)

pd.set_option('display.max_columns', None)
print(f"df_original: {type(df_original)} shape={df_original.shape}\n{df_original}")                         # <class 'pandas.core.frame.DataFrame'> shape=(100, 8)
pd.reset_option('display.max_columns')


dataset: <class 'datasets.arrow_dataset.Dataset'> shape=(1000, 149)
Dataset({
    features: ['id', 'text', '142', '143', '146', '147', '149', '150', '152', '154', '155', '156', '158', '160', '162', '165', '167', '169', '170', '171', '173', '174', '175', '176', '356', '360', '361', '362', '364', '370', '371', '373', '375', '376', '394', '408', '667', '686', '687', '689', '756', '758', '760', '761', '900', '901', '902', '903', '905', '906', '907', '908', '909', '911', '912', '913', '914', '916', '263', '265', '266', '267', '268', '270', '271', '272', '273', '274', '275', '276', '277', '278', '279', '283', '284', '285', '286', '287', '288', '289', '291', '292', '293', '297', '298', '299', '300', '301', '302', '303', '304', '358', '359', '365', '367', '369', '379', '382', '385', '409', '610', '611', '613', '614', '615', '618', '619', '637', '668', '670', '672', '673', '676', '677', '678', '681', '682', '683', '810', '811', '812', '813', '814', '815', '816', '817', '818', '819', '820', '821

##### Create labels (list), id2label (dict) and label2id (dict).
**The sequence of the labels list is the same as in dataset.
And the sequences of the optimized thresholds, true labels and predictions are preserved.**

* dataset 7_1000_125_125  ,  48 labels
* dataset 7_128_18_54     ,  42 labels
* dataset 8910_1087_68_204, 206 labels
* dataset 11_1000         ,   6 labels

In [12]:
labels = [label for label in dataset.features.keys() if label not in ['id', 'text']]
#labels.sort()

print(f"labels: {type(labels)} {len(labels)}\n{labels}")

num_labels = len(labels)

labels: <class 'list'> 147
['142', '143', '146', '147', '149', '150', '152', '154', '155', '156', '158', '160', '162', '165', '167', '169', '170', '171', '173', '174', '175', '176', '356', '360', '361', '362', '364', '370', '371', '373', '375', '376', '394', '408', '667', '686', '687', '689', '756', '758', '760', '761', '900', '901', '902', '903', '905', '906', '907', '908', '909', '911', '912', '913', '914', '916', '263', '265', '266', '267', '268', '270', '271', '272', '273', '274', '275', '276', '277', '278', '279', '283', '284', '285', '286', '287', '288', '289', '291', '292', '293', '297', '298', '299', '300', '301', '302', '303', '304', '358', '359', '365', '367', '369', '379', '382', '385', '409', '610', '611', '613', '614', '615', '618', '619', '637', '668', '670', '672', '673', '676', '677', '678', '681', '682', '683', '810', '811', '812', '813', '814', '815', '816', '817', '818', '819', '820', '821', '822', '823', '824', '825', '826', '827', '828', '829', '830', '135', '390',

In [13]:
id2label = {idx: label for idx, label in enumerate(labels)}
print(f"id2label: {type(id2label)} {len(id2label)}\n{id2label}")

id2label: <class 'dict'> 147
{0: '142', 1: '143', 2: '146', 3: '147', 4: '149', 5: '150', 6: '152', 7: '154', 8: '155', 9: '156', 10: '158', 11: '160', 12: '162', 13: '165', 14: '167', 15: '169', 16: '170', 17: '171', 18: '173', 19: '174', 20: '175', 21: '176', 22: '356', 23: '360', 24: '361', 25: '362', 26: '364', 27: '370', 28: '371', 29: '373', 30: '375', 31: '376', 32: '394', 33: '408', 34: '667', 35: '686', 36: '687', 37: '689', 38: '756', 39: '758', 40: '760', 41: '761', 42: '900', 43: '901', 44: '902', 45: '903', 46: '905', 47: '906', 48: '907', 49: '908', 50: '909', 51: '911', 52: '912', 53: '913', 54: '914', 55: '916', 56: '263', 57: '265', 58: '266', 59: '267', 60: '268', 61: '270', 62: '271', 63: '272', 64: '273', 65: '274', 66: '275', 67: '276', 68: '277', 69: '278', 70: '279', 71: '283', 72: '284', 73: '285', 74: '286', 75: '287', 76: '288', 77: '289', 78: '291', 79: '292', 80: '293', 81: '297', 82: '298', 83: '299', 84: '300', 85: '301', 86: '302', 87: '303', 88: '304', 8

In [14]:
label2id = {label: idx for idx, label in enumerate(labels)}
print(f"label2id: {type(label2id)} {len(label2id)}\n{label2id}")

label2id: <class 'dict'> 147
{'142': 0, '143': 1, '146': 2, '147': 3, '149': 4, '150': 5, '152': 6, '154': 7, '155': 8, '156': 9, '158': 10, '160': 11, '162': 12, '165': 13, '167': 14, '169': 15, '170': 16, '171': 17, '173': 18, '174': 19, '175': 20, '176': 21, '356': 22, '360': 23, '361': 24, '362': 25, '364': 26, '370': 27, '371': 28, '373': 29, '375': 30, '376': 31, '394': 32, '408': 33, '667': 34, '686': 35, '687': 36, '689': 37, '756': 38, '758': 39, '760': 40, '761': 41, '900': 42, '901': 43, '902': 44, '903': 45, '905': 46, '906': 47, '907': 48, '908': 49, '909': 50, '911': 51, '912': 52, '913': 53, '914': 54, '916': 55, '263': 56, '265': 57, '266': 58, '267': 59, '268': 60, '270': 61, '271': 62, '272': 63, '273': 64, '274': 65, '275': 66, '276': 67, '277': 68, '278': 69, '279': 70, '283': 71, '284': 72, '285': 73, '286': 74, '287': 75, '288': 76, '289': 77, '291': 78, '292': 79, '293': 80, '297': 81, '298': 82, '299': 83, '300': 84, '301': 85, '302': 86, '303': 87, '304': 88, '

##### Download the tokenizer and the model

In [15]:
print("Tokenizer")
tokenizer = LongformerTokenizerFast.from_pretrained(repo_id, timeout=60)  # Increased timeout to 60 seconds)

print("Model")
model = LongformerForSequenceClassification.from_pretrained(repo_id)
model.eval()  # Ensures no gradient computation

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


Tokenizer


tokenizer_config.json:   0%|          | 0.00/1.28k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.56M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

Model


config.json:   0%|          | 0.00/5.61k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/595M [00:00<?, ?B/s]

LongformerForSequenceClassification(
  (longformer): LongformerModel(
    (embeddings): LongformerEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
      (position_embeddings): Embedding(4098, 768, padding_idx=1)
    )
    (encoder): LongformerEncoder(
      (layer): ModuleList(
        (0-11): 12 x LongformerLayer(
          (attention): LongformerAttention(
            (self): LongformerSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (query_global): Linear(in_features=768, out_features=768, bias=True)
              (key_global): Linear(in_features=768, out_features=768, bias=True)
          

##### label: count: threshold

In [16]:
# Step 1: Merge counts and thresholds into one dict
combined_dict = {
    label: {
        "count": TrEvTe_by_count_gt_0[label],
        "thresh": optimized_thresholds_dict[label]
    }
    for label in TrEvTe_by_count_gt_0
    if label in optimized_thresholds_dict
}

print(f"combined_dict: {type(combined_dict)} len={len(combined_dict)}\n{combined_dict}")
for label, v in combined_dict.items():
    print(f"{label}:\t{v['thresh']:.2f}\t{v['count']}")

# Step 2: Sort by count desc
#sorted_combined_dict = dict(sorted(combined_dict.items(), key=lambda item: item[1]["count"], reverse=True))


#raise Exception("I stop here")

combined_dict: <class 'dict'> len=147
{'138': {'count': {'count': 45021, 'Value': 'EXPERIENCE_MORE_THAN_5_YEARS', 'ShortValue': '5y-10y'}, 'thresh': 0.44999999999999996}, '139': {'count': {'count': 38840, 'Value': 'EXPERIENCE_MORE_THAN_10_YEARS', 'ShortValue': '>10y'}, 'thresh': 0.49999999999999994}, '137': {'count': {'count': 35796, 'Value': 'EXPERIENCE_2_TO_5_YEARS', 'ShortValue': '2y-5y'}, 'thresh': 0.49999999999999994}, '352': {'count': {'count': 17701, 'Value': 'High school', 'ShortValue': None}, 'thresh': 0.44999999999999996}, '142': {'count': {'count': 16712, 'Value': 'Full-stack Developer', 'ShortValue': None}, 'thresh': 0.49999999999999994}, '136': {'count': {'count': 11057, 'Value': 'EXPERIENCE_6_MONTHS_TO_2_YEARS', 'ShortValue': '6m-2y'}, 'thresh': 0.44999999999999996}, '149': {'count': {'count': 10153, 'Value': 'Functional Analyst', 'ShortValue': None}, 'thresh': 0.44999999999999996}, '160': {'count': {'count': 7333, 'Value': 'System Engineer', 'ShortValue': None}, 'thresh'

##### Increasing optimized_thresholds too low because they capture noise

- count_0_list = ["900", "901", "902", "903", "905", "907", "909", "911", "912", "913", "914", "916"]
  - count = 0 in prediction below
  - count in trEvTE below:    
        "900": 3,
        "901": 2,
        "902": 1,
        "903": 7,
        "905": 5,
        "907": 2,
        "908": 5,
        "909": 1,
        "911": 6,
        "912": 1,
        "913": 5,
        "914": 3,
        "916": 4,
- count_01_025_list = ["908", "913", "672", "673"]
  - first normalized_thresholds 908 and 913: 0.1, 672 and 673: 0.25
  - count in prediction below: 908: 1, 913: 0, 672: 3, 673: 1
  - count in trEvTe below: 908: 5, 913: 5, 672: 37, 673: 41


In [17]:
print(f"optimized_thresholds_dict: {type(optimized_thresholds_dict)} len={len(optimized_thresholds_dict)}\n{optimized_thresholds_dict}")
print()

optimized_thresholds_model = np.array(list(optimized_thresholds_dict.values()))
print(f"optimized_thresholds_model: {type(optimized_thresholds_model)} shape={optimized_thresholds_model.shape}\n{optimized_thresholds_model}")

#count_0_list = ["900", "901", "902", "903", "905", "907", "909", "911", "912", "913", "914", "916"]
#count_01_025_list = ["908", "913", "672", "673"]
#count_both = count_0_list + count_01_025_list

#rare_labels_in_trEvTe = [label for label, count in trEvTe_counter_dict.items() if count < min_trEvTe_count]
#print(f"rare_labels_in_trEvTe: {type(rare_labels_in_trEvTe)} len={len(rare_labels_in_trEvTe)}\n{rare_labels_in_trEvTe}")

#for idx, (label, count) in enumerate(Pr_all_count_by_count_dict.items()):
#    print(f"{idx}: {label} -> {count}: {optimized_thresholds[idx]:.2f}")
#    #if label in count_both:
#    if label in rare_labels_in_trEvTe:
#        optimized_thresholds[idx] = min_threshold
#for idx, (label, count) in enumerate(TrEvTe_all_count_by_count_gt_0_dict.items()):
#    print(f"{idx}: {label} -> {count}: {optimized_thresholds[idx]:.2f}")

print("idx:\tlabel -> count\t optimized_threshold_model Value:")
for idx, (label, v) in enumerate(TrEvTe_by_SkillTypeIdComplexityId_gt_0.items()):
    print(f"{idx}:\t{label} -> {v['count']}\t{optimized_thresholds_model[idx]:.2f}\t({v['Value']})")
print()

# Vectorize update of optimized_thresholds
low_counts = [(idx, label, v["count"]) for idx, (label, v) in enumerate(TrEvTe_by_SkillTypeIdComplexityId_gt_0.items()) if v["count"] < min_trEvTe_count]

print("low_counts: index: label -> count\toptimized_threshold_model:")
for (idx, label, v) in low_counts:
    print(f"{idx}:\t{label} -> {v}\t{optimized_thresholds_model[idx]:.2f}")

low_count_indices = [idx for idx, _, _ in low_counts]
print(f"low count indices: {type(low_count_indices)} len={len(low_count_indices)}\n{low_count_indices}")

optimized_thresholds_modified                    = optimized_thresholds_model.copy()
optimized_thresholds_modified[low_count_indices] = min_threshold
print(f"optimized_thresholds_modified : {type(optimized_thresholds_modified)} shape={optimized_thresholds_modified.shape}\n{optimized_thresholds_modified}")
for (idx, label, count) in low_counts:
    print(f"{idx}:\t{label} -> {count}:\t{optimized_thresholds_model[idx]:.2f} -> {optimized_thresholds_modified[idx]:.2f}")

optimized_thresholds = optimized_thresholds_modified
print(f"optimized_thresholds: {type(optimized_thresholds)} shape={optimized_thresholds.shape}\n{optimized_thresholds}")

#raise Exception("I stop here")

optimized_thresholds_dict: <class 'dict'> len=147
{'142': 0.49999999999999994, '143': 0.44999999999999996, '146': 0.44999999999999996, '147': 0.44999999999999996, '149': 0.44999999999999996, '150': 0.49999999999999994, '152': 0.39999999999999997, '154': 0.49999999999999994, '155': 0.49999999999999994, '156': 0.44999999999999996, '158': 0.44999999999999996, '160': 0.44999999999999996, '162': 0.44999999999999996, '165': 0.49999999999999994, '167': 0.44999999999999996, '169': 0.44999999999999996, '170': 0.44999999999999996, '171': 0.44999999999999996, '173': 0.39999999999999997, '174': 0.49999999999999994, '175': 0.49999999999999994, '176': 0.49999999999999994, '356': 0.49999999999999994, '360': 0.5499999999999999, '361': 0.39999999999999997, '362': 0.39999999999999997, '364': 0.44999999999999996, '370': 0.39999999999999997, '371': 0.44999999999999996, '373': 0.39999999999999997, '375': 0.39999999999999997, '376': 0.44999999999999996, '394': 0.44999999999999996, '408': 0.44999999999999996

In [18]:
def format_int_or_NaN(x):
    return int(x) if not np.isnan(x) else "NaN"

In [19]:
def calculate_confusion_multi(precision, recall, support, total_samples):
    precision     = np.array(precision)
    recall        = np.array(recall)
    support       = np.array(support)
    total_samples = np.array(total_samples)

    with np.errstate(divide='ignore', invalid='ignore'):
        f1_score = 2 * (precision * recall) / (precision + recall)
        TP       = recall * support
        FN       = support - TP
        FP       = TP * (1 - precision)/precision
        TN       = total_samples - TP - FN - FP

    return pd.DataFrame({
        'TP' : TP,
        'FN' : FN,
        'FP' : FP,
        'TN' : TN
    })

In [20]:
def analyze_multi_label_performance(y_true, y_pred, label_names, plot=True):
    """
    Returns:
    - label_df    : per-label precision, recall, f1, support (DataFrame)
    - summary_df  : micro, macro, weighted and samples averages (DataFrame)
    - confusion_df: per-label TP, FP, FN, TN (DataFrame)
    """
    report_dict = classification_report(
        y_true, y_pred,
        target_names=label_names,
        zero_division="warn",
        output_dict=True
    )
    report_df = pd.DataFrame(report_dict).T
    #print(f"report_df: {type(report_df)} shape={report_df.shape}\n{report_df}")

    # label_df
    label_df = report_df.loc[label_names].copy()
    label_df['support'] = label_df['support'].apply(format_int_or_NaN)
    #print(f"label_df: {type(label_df)} shape={label_df.shape}\n{label_df}")

    precision = label_df['precision'].values
    recall    = label_df['recall'].values
    f1_score  = label_df['f1-score'].values
    support   = label_df['support'].values

    total_samples = len(y_true)
    print(f"total_samples: {type(total_samples)} {total_samples}")  # <class 'int'> 1000
    total_support = label_df['support'].sum()
    print(f"total_support: {type(total_support)} {total_support}")  # <class 'numpy.float64'> 6441.0

    confusion_df = calculate_confusion_multi(precision, recall, support, total_samples)
    #confusion_df.index = label_names
    confusion_df = confusion_df.set_index(pd.Index(label_names))
    #confusion_df       = confusion_df.applymap(format_int_or_NaN)
    confusion_df = confusion_df.apply(lambda col: col.map(format_int_or_NaN))
    #print(f"confusion_df: {type(confusion_df)} shape={confusion_df.shape}\n{confusion_df}")

    # All average rows: micro, macro, weighted, samples
    avg_rows      = ["micro avg", "macro avg", "weighted avg", "samples avg"]
    existing_avgs = [row for row in avg_rows if row in report_df.index]

    summary_df            = report_df.loc[existing_avgs, ["precision", "recall", "f1-score", "support"]]
    summary_df.loc["micro avg", "support"]    = total_support
    summary_df.loc["macro avg", "support"]    = np.nan         # no clear meaning
    summary_df.loc["weighted avg", "support"] = total_support  # number of samples
    summary_df.loc["samples avg", "support"]  = total_samples  # number of samples
    summary_df["support"]                     = summary_df["support"].apply(format_int_or_NaN)
    #print(f"summary_df: {type(summary_df)} shape={summary_df.shape}\n{summary_df}"


    # Optional plot
    if plot:
        #label_df[["precision", "recall", "f1", "support"]].plot(
        label_df[["precision", "recall", "support"]].plot(
            kind="bar", figsize=(10,6), ylim=(0,1),
            title="Per-Label Precision Recall F1 Support"
        )
        plt.grid(axis='y')
        plt.ylabel("Score")
        plt.tight_layout()
        plt.show()

    return label_df, confusion_df, summary_df



## case TP=0 FN!=0 and case TP=0 FN=0

|label|TP|FN|FP|TN|label|precision|recall|f1|support|
|:---:|::|::|::|::|:---:|:-------:|:----:|::|:-----:|
|266|0|3|NaN|NaN|266|0.00|0.00|0.00|3|
|267|0|2|NaN|NaN|267|0.00|0.00|0.00|2|
|273|0|3|NaN|NaN|273|0.00|0.00|0.00|3|
|275|0|0|NaN|NaN|275|0.00|0.00|0.00|0|
|279|0|1|NaN|NaN|279|0.00|0.00|0.00|1|
|285|0|3|NaN|NaN|285|0.00|0.00|0.00|3|
|301|0|0|NaN|NaN|301|0.00|0.00|0.00|0|
|618|0|0|NaN|NaN|618|0.00|0.00|0.00|0|
|681|0|3|NaN|NaN|681|0.00|0.00|0.00|3|
|815|0|1|NaN|NaN|815|0.00|0.00|0.00|1|
|825|0|1|NaN|NaN|825|0.00|0.00|0.00|1|

- TP=0 and FN!=0:
 - precision = 0
 - recall    = 0
 - f1        = 0
 - support   = FN (some true positives to be found)
- TP=FN=0:
 - precision = 0/0 (undefined)
 - recall    = 0/0 (undefined)
 - f1        = 0/0 (undefined)
 - support   = 0   (label never present in the ground truth, the model never predicted it)
- Check using jobs_preprocessing_CounterPrediction:
 - count_275: <class 'int'> 0
 - count_301: <class 'int'> 0
 - count_618: <class 'int'> 0

In [21]:
def export_filtered_predictions(
        df: pd.DataFrame,
        output_txt: str
):
    label_columns = [col for col in df.columns if col not in ["id", "text", "true_labels", "preds", "compare"]]

    with open(output_txt, 'w', encoding="utf-8") as f:
        for i, row in df.iterrows():
            true_labels = row["true_labels"]
            preds       = row["preds"]

            # Skip if both true and predicted are all False
            if not any(true_labels) and not any(preds):
                continue

            f.write(f"id: {row['id']}\n")
            f.write(f"text: {row['text']}\n")

            if row["compare"] == "OK":
                # Just print active labels for both
                true_active  = [label for label, val in zip(label_columns, true_labels) if val]
                pred_active  = [label for label, val in zip(label_columns, preds)       if val]
                f.write(f"true labels     : {' '.join(true_active)}\n")
                f.write(f"predicted labels: {' '.join(pred_active)}\n")
            else:
                # Align the labels for visual comparison
                aligned_true = []
                aligned_pred = []
                for label, t_val, p_val in zip(label_columns, true_labels, preds):
                    if t_val or p_val:
                        aligned_true.append(label if t_val else ' '*len(label))
                        aligned_pred.append(label if p_val else ' '*len(label))
                f.write("true labels:      " + ' '.join(aligned_true) + '\n')
                f.write("predicted labels: " + ' '.join(aligned_pred) + '\n')

            f.write(f"compare: {row['compare']}\n\n")

        print(f"Saved aligned results to {output_txt}")


In [22]:
# Step 1: Tokenize dataset
def tokenize_function(examples):
  return tokenizer(examples["text"], padding='max_length', max_length=max_length, truncation=True)

tokenized_dataset = dataset.map(tokenize_function, batched=True)
# <class 'datasets.arrow_dataset.Dataset'> shape=(100, 10)
#print(f"tokenized_dataset: {type(tokenized_dataset)} shape={tokenized_dataset.shape}\n{tokenized_dataset}")

tokenized_df = tokenized_dataset.to_pandas()
#print(f"tokenized_df: {type(tokenized_df)} shape={tokenized_df.shape}\n{tokenized_df}")

# Step 2: Convert to PyTorch DataLoader
tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"] + labels)
dataloader = DataLoader(tokenized_dataset, batch_size=batch_size)

# Step 3: Run Prediction
all_preds = []

with torch.no_grad():                       # No gradients needed for prediction
    for batch in dataloader:
        # Move batch to GPU if available
        inputs = {k: batch[k].to(device) for k in ["input_ids", "attention_mask"]}

        # Forward pass
        outputs = model(**inputs)
        logits = outputs.logits  # Model outputs raw logits

        # Apply sigmoid to convert logits into probabilities
        probs = torch.sigmoid(logits)

        # Move probs to CPU and convert to NumPy
        all_preds.extend(probs.cpu().numpy())

all_preds_arr = np.array(all_preds)
#print(f"all_preds    : {type(all_preds)}     len={len(all_preds)}\n{all_preds}")
print(f"all_preds_arr: {type(all_preds_arr)} shape={all_preds_arr.shape}\n{all_preds_arr}")
print(f"all_preds_arr[0]: {all_preds_arr[0]}")

# Step 4: Convert Probabilities to Binary Predictions
if threshold_tuning:
    thresholds = optimized_thresholds
else:
    thresholds = np.full(num_labels, 0.5)
print(f"thresholds: {type(thresholds)} shape={thresholds.shape}\n{thresholds}")                  # <class 'numpy.ndarray'> shape=(6,)
print()

binary_preds = (np.array(all_preds) > thresholds).astype(int)
#print(f"binary_preds: {type(binary_preds)} shape={binary_preds.shape}\n{binary_preds}")          # <class 'numpy.ndarray'> shape=(100, 6)
print(f"binary_preds[0]: {binary_preds[0]}")

# Step 5: Compare with True Labels
true_labels = tokenized_df[labels]
#print(f"true_labels: {type(true_labels)} shape={true_labels.shape}\n{true_labels}")              # <class 'pandas.core.frame.DataFrame'> shape=(100, 6)

# Convert true_labels DataFrame to a NumPy array of 0 and 1
true_labels_np = true_labels.astype(int).to_numpy()
#print(f"true_labels_np: {type(true_labels_np)} shape={true_labels_np.shape}\n{true_labels_np}")  # <class 'numpy.ndarray'> shape=(100, 6)

# Convert id2label dict to a list of label names
label_names = list(id2label.values())
print(f"label_names: {type(label_names)} len={len(label_names)}\n{label_names}")  # <class 'list'> len=126
print()

#classification_report(y_true, y_pred)
report = classification_report(
    true_labels_np, binary_preds, target_names=label_names, zero_division="warn")
print(f"classification report:{type(report)} len={len(report)}\n{report}")      # <class 'str'> len=7076
print()

#report2 = classification_report(true_labels_np, binary_preds, target_names=label_names, zero_division="warn", output_dict=True)
#print(f"classification report2:{type(report2)} len={len(report2)}\n{report2}")  # <class 'dict'> len=130
#print()

label_df, confusion_df, summary_df = analyze_multi_label_performance(
    true_labels_np, binary_preds, label_names=label_names, plot=False)

label_df[['precision', 'recall', 'f1-score']] = label_df[['precision', 'recall', 'f1-score']].round(2)
label_df['support'] = label_df['support'].astype(int)

summary_df[['precision', 'recall', 'f1-score']] = summary_df[['precision', 'recall', 'f1-score']].round(2)

print("__________________________________________")

print("\nclassification report (= label_df):\n", label_df)

print("__________________________________________")

print("\nAll averages:\n", summary_df)

print("__________________________________________")

#pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
#pd.set_option('display.max_colwidth', None)
print("\nConfusion matrix:")
print("precision = TP / (TP+FP)")
print("recall    = TP / (TP+FN)")
print("1/f1      = 1/2 * (1/precision + 1/recall")
print("support   = TP+FN")
print("\nConfusion matrix:\n", confusion_df)

print("__________________________________________")

df_compare                = df_original.copy()
df_compare['true_labels'] = true_labels.values.tolist()
df_compare['preds']       = binary_preds.tolist()
df_compare['compare']     = (df_compare['true_labels']==df_compare['preds']).replace({True:'OK', False:'KO'})
#print(f"df_compare: {type(df_compare)} shape={df_compare.shape}\n{df_compare}")  # <class 'pandas.core.frame.DataFrame'>
print()

def print_row(row, file):
    """
    Prints the row information to the console and writes it to a file.
    """
    output  = f"id: {row['id']}\n"
    output += f"text: {row['text']}\n"
    output += "labels     : [" + ", ".join(f"{label:>5}" for label in labels) + "]\n"
    output += "true_labels: [" + ", ".join(f"{true_label:>5}" for true_label in row['true_labels']) + "]\n"
    output += "preds      : [" + ", ".join(f"{pred:>5}" for pred in row['preds']) + "]\n"
    output += f"compare    : {row['compare']}\n"
    output += "\n"

    print(output, end="")  # Print to console without extra newline
    file.write(output)     # Write to file

    return None # Return something

# Open the file in write mode
#with open(compare_true_labels_preds_file, "w") as f:
#    _ = df_compare.apply(print_row, axis=1, args=(f,))  # Pass file object to print_row

# Download the file
#files.download(compare_true_labels_preds_file)

export_filtered_predictions(df_compare, output_txt)
print("coucou")
# Count total 'OK' and 'KO' values
total_ok = (df_compare['compare'] == 'OK').sum()  # Total matches
total_ko = (df_compare['compare'] == 'KO').sum()  # Total mismatches
print(f"Total 'OK': {total_ok}")
print(f"Total 'KO': {total_ko}")
print()

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Initializing global attention on CLS token...


all_preds_arr: <class 'numpy.ndarray'> shape=(1000, 147)
[[0.22002117 0.21013895 0.30684862 ... 0.34341758 0.50139976 0.32064   ]
 [0.25732878 0.15306309 0.15196231 ... 0.31022003 0.63741285 0.30747128]
 [0.3175166  0.13576026 0.33978316 ... 0.28895012 0.4685333  0.34445435]
 ...
 [0.52471656 0.16368032 0.49445516 ... 0.29479697 0.6456696  0.244556  ]
 [0.5435394  0.48522958 0.19912437 ... 0.3781013  0.6573637  0.22146086]
 [0.65333176 0.32439864 0.3424744  ... 0.35773522 0.55716616 0.375499  ]]
all_preds_arr[0]: [0.22002117 0.21013895 0.30684862 0.2199176  0.2811636  0.12642495
 0.19332525 0.16467126 0.17117693 0.21663375 0.20632268 0.21807219
 0.16114528 0.15519787 0.17517449 0.24425657 0.3341609  0.24262147
 0.19362484 0.2536057  0.30132914 0.58215237 0.26659498 0.41264418
 0.16820058 0.20244943 0.2265754  0.51349217 0.15480527 0.17050067
 0.24679838 0.19809765 0.15036583 0.23648076 0.25388545 0.19331123
 0.27105796 0.1712227  0.14771494 0.53081614 0.22504558 0.13507417
 0.06483287 

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/me

In [23]:
print("It's the end")

It's the end


In [24]:
raise Exception("I stop here")

Exception: I stop here